In [10]:
import numpy
from tqdm import tqdm
import pickle
from src.load_data import load_data_and_drop_duplicated
from src.slope_calculation import (
    baseline_slope_calculation,
    get_rdp_points,
    get_interpolated_rdp_slope
)

In [3]:
def load_data():
    df = load_data_and_drop_duplicated("../data/Transects_Level_2_LBRUT.shp")
    return df

def get_pk_and_elevation_array(df):
    elevation_array = numpy.array(df["LB_Q25_COR"])
    pk_array = numpy.array(df["PK"])
    return pk_array, elevation_array

In [11]:
df = load_data()
pk_array, elevation_array = get_pk_and_elevation_array(df)
data_dict = {}
data_dict["pk_array"] = pk_array
data_dict["elevation_array"] = elevation_array
data_dict["baseline_slope"] = baseline_slope_calculation(pk_array, elevation_array)
data_dict["rdp_epsilon"] = {}
for epsilon in tqdm(numpy.arange(0,1.01,0.01)):
    epsilon = round(epsilon, 2)
    rdp_points_kept_array = get_rdp_points(pk_array, elevation_array, epsilon)
    rpd_slope_interpolation = get_interpolated_rdp_slope(pk_array, rdp_points_kept_array)
    data_dict["rdp_epsilon"][epsilon] = {
        "rdp_points_kept_array": rdp_points_kept_array,
        "rpd_slope_interpolation": rpd_slope_interpolation
    }

100%|██████████| 101/101 [05:31<00:00,  3.28s/it]


In [26]:
with open ("data/data_dict.pkl", "wb") as f:
    pickle.dump(data_dict, f)